# Experimento para detección semi-automática de recorridos, tomando agrupando por idsam + fecha

La idea es que utilizando el patrón dibujado por cada idsam dentro de un día se pueda detectar y agrupar los patrones de recorridos iguales/similares a manera de poder detectar la linea/ramal pudiendo ignorar el idrutaestacion.

Esto nos permitiria:
a) Auto-detectar el ramal por su trayectoria y poder empezar a limpiar el idrutaestacion 0000
b) Detectar alteraciones y anormalidades en el trayecto esperado.
c) Predecir congestión? Acá ya volando alto, pero el markdown perdona todo.

Importar la librerías que vamos a necesitar. Como vamos a verificar mucho del funcionamiento manualmente para poder desarrollar una metodologia, vamos a hacer varios mapas.

In [12]:
import pandas as pd
import numpy as np

import os
import datetime

import geopandas as gpd
import contextily as cx

import matplotlib.pyplot as plt

Carguemos los datos de una linea de prueba, para este caso vamos a usar la idrutaestacion __0145__ que esta asociada a __"ITA - LAMBARÉ (Canal 13) Troncal 1"__ de la empresa **"3 DE FEBRERO SA"**

In [13]:
datos_rutas = pd.read_csv("../../data/interim/datosporlinea/linea-0145.csv", sep = ';',low_memory=False)
print(datos_rutas.sample(10))

                            serialtarjeta           idsam   
669965   2adf954368da07404dfeeff7dc89a18a  04081e1a846280  \
881627   60b9a05b937074dad85137a2d2eab766  0411171aaa6380   
964450   679c37563fe2f381463fde0ed9fcd7f4  0408131a846280   
236829   6dd42b2b94fe12a125a0df29b0f8d090  04301622b95a80   
296378   8e9b56f0bfe90eaaf9f3cb3f6997d63b  041c20e24e6180   
883085   17b11381d39ff8e7826b6e86ab4ecc6a  0408211a846280   
588475   9d1ec327d38e1888f62585c05af8b9e4  0408211a846280   
361906   cdfed57ad1661aac564b41774ac12905  0408181a846280   
1003841  4a27c427a927df0297ac5d927294a2ad  0411171aaa6380   
746718   2d294cc54eba2fca9f9c1c2fea438e95  04081b1a846280   

                       fechahoraevento producto  montoevento   
669965   2022/04/12 15:33:58.000000000       MO         3400  \
881627   2022/07/13 12:27:00.000000000       MO         2300   
964450   2022/08/11 06:20:02.000000000       MO         2300   
236829   2022/11/18 09:25:40.000000000       MO         3400   
296378  

Ahora empezamos a limpiar todos los datos que no nos interesan.

In [14]:
print(datos_rutas.columns.values.tolist())

['serialtarjeta', 'idsam', 'fechahoraevento', 'producto', 'montoevento', 'consecutivoevento', 'identidad', 'tipoevento', 'latitude', 'longitude', 'idrutaestacion', 'tipotransporte']


In [15]:
datos_rutas.drop(['serialtarjeta', 'producto', 'montoevento', 'consecutivoevento', 'identidad', 'tipoevento', 'tipotransporte'],inplace=True, axis=1)
print(datos_rutas.sample(10))

                 idsam                fechahoraevento  latitude  longitude   
307583  04132822626c80  2022/12/20 16:49:56.000000000 -25.50757  -57.35747  \
816086  0408171a846280  2022/06/07 12:54:10.000000000 -25.50931  -57.35830   
341485  0408151a846280  2022/12/07 12:45:56.000000000 -25.31231  -57.58631   
66094   0408171a846280  2022/02/24 10:01:00.000000000 -25.31360  -57.58713   
486011  04081b1a846280  2023/02/02 18:44:12.000000000 -25.50482  -57.35738   
879814  0408201a846280  2022/07/13 21:00:54.000000000 -25.33426  -57.53628   
882362  041c1ce24e6180  2022/07/13 08:30:30.000000000 -25.35629  -57.61495   
582449  04301a22b95a80  2022/03/24 13:39:02.000000000 -25.32474  -57.59633   
95161   041a38e24e6180  2022/02/11 18:17:54.000000000 -25.47178  -57.41839   
844220  04132f1aaa6380  2022/07/27 17:45:12.000000000 -25.41707  -57.45640   

        idrutaestacion  
307583             145  
816086             145  
341485             145  
66094              145  
486011          

Ahora que solamente tenemos los identificadores de los lectores (buses), los momentos en que ocurren los eventos, la posición geografica del evento (latitude,longitude) y el identificador de la ruta; ya podemos empezar a trabajar.

In [16]:
datos_rutas.info()
datos_rutas.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074281 entries, 0 to 1074280
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   idsam            1074281 non-null  object 
 1   fechahoraevento  1074281 non-null  object 
 2   latitude         1074281 non-null  float64
 3   longitude        1074281 non-null  float64
 4   idrutaestacion   1074281 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 41.0+ MB


,idsam,fechahoraevento,latitude,longitude,idrutaestacion
348995,041d0fe24e6180,2022/12/03 19:50:02.000000000,-25.51014,-57.36138,145
724380,041918e24e6180,2022/05/17 08:25:20.000000000,-25.50701,-57.35778,145
357419,0408151a846280,2022/12/01 00:01:26.000000000,-25.33392,-57.53706,145
396534,041922e24e6180,2023/01/15 15:17:12.000000000,-25.32485,-57.59638,145
90037,04301622b95a80,2022/02/14 06:17:52.000000000,-25.33890,-57.58467,145
610853,041d12e24e6180,2022/03/10 09:39:44.000000000,-25.33411,-57.61617,145
286388,041c1de24e6180,2022/12/29 18:46:38.000000000,-25.31616,-57.57523,145
798656,0411171aaa6380,2022/06/13 07:50:26.000000000,-25.34902,-57.51281,145
904648,041d12e24e6180,2022/07/04 13:47:50.000000000,-25.31274,-57.58659,145
301703,04300e22b95a80,2022/12/22 20:03:12.000000000,-25.31756,-57.57156,145


Primeramente revisamos los datos y hacemos algo de limpieza y ajustes.

In [17]:
# Convertimos la columna de cadena a fecha
datos_rutas['fechahoraevento']= pd.to_datetime(datos_rutas['fechahoraevento'])

# Remover todos los casos donde latitud y longitud sean ceros
datos_rutas = datos_rutas[(datos_rutas[['latitude','longitude']] != 0).all(axis=1)] 
# datos_rutas["fechastr"] = datos_rutas['fechahoraevento'].dt.strftime('%Y%m%d%H%M')
datos_rutas["fechastr"] = datos_rutas['fechahoraevento'].dt.strftime('%Y%m%d')
# REDUCIMOS LA RESOLUCION DE LOS DATOS GPS A 4 DECIMALES PARA FACILITAR LA AGRUPACION Y GENERACION DE TRAYECTORIAS
datos_rutas["latitude"]=datos_rutas["latitude"] * 1.000000000001
datos_rutas["longitude"]=datos_rutas["longitude"] * 1.000000000001

# datos_rutas.loc[:, "latitude"] = datos_rutas["latitude"].map('{:.4f}'.format)
# datos_rutas.loc[:, "longitude"] = datos_rutas["longitude"].map('{:.4f}'.format)

datos_rutas["latitude"]=datos_rutas["latitude"].round(4) 
datos_rutas["longitude"]=datos_rutas["longitude"].round(4)
datos_rutas

,idsam,fechahoraevento,latitude,longitude,idrutaestacion,fechastr
0,041d12e24e6180,2022-01-30 23:49:56,-25.3895,-57.4701,145,20220130
1,041d12e24e6180,2022-01-30 23:40:46,-25.3455,-57.5054,145,20220130
2,041d12e24e6180,2022-01-30 23:40:40,-25.3455,-57.5054,145,20220130
3,041d12e24e6180,2022-01-30 23:40:34,-25.3455,-57.5054,145,20220130
4,041d12e24e6180,2022-01-30 23:40:30,-25.3455,-57.5054,145,20220130
...,...,...,...,...,...,...
1074276,04132822626c80,2022-09-01 05:22:52,-25.3420,-57.6249,145,20220901
1074277,04132822626c80,2022-09-01 05:22:22,-25.3422,-57.6263,145,20220901
1074278,04132822626c80,2022-09-01 05:20:38,-25.3427,-57.6333,145,20220901
1074279,04132822626c80,2022-09-01 05:20:38,-25.3427,-57.6333,145,20220901


Ahora vamos a tomar un idsam en particular y vamos a comenzar el trabajo de intentar crear una ruta estandar contra la cual vamos a comprar a las demas.

In [18]:
# Creamos un listado de todos los idsams (lectores-vehiculos) presentes en el dataset limpio
idsams = datos_rutas['idsam'].unique() 
print(idsams.shape)
print(idsams)
fechas_trayectorias = datos_rutas['fechastr'].unique() 
print(fechas_trayectorias.shape)
print(fechas_trayectorias)

(45,)
['041d12e24e6180' '04132822626c80' '04300e22b95a80' '0408191a846280'
 '04081b1a846280' '041922e24e6180' '04132f1aaa6380' '0408181a846280'
 '041c24e24e6180' '0408131a846280' '04191be24e6180' '041a38e24e6180'
 '041c21e24e6180' '04301222b95a80' '041c1ce24e6180' '041d14e24e6180'
 '041c20e24e6180' '04301522b95a80' '0408211a846280' '0413271aaa6380'
 '04081f1a846280' '0408171a846280' '0408201a846280' '04191fe24e6180'
 '041d0fe24e6180' '04081e1a846280' '04301722b95a80' '04082c1a846280'
 '0411171aaa6380' '04301622b95a80' '0408151a846280' '0409131a846280'
 '041917e24e6180' '041c1fe24e6180' '041c22e24e6180' '041c1de24e6180'
 '041918e24e6180' '04121b1aaa6380' '04131122626c80' '04301a22b95a80'
 '04303e22b95a80' '04120e1aaa6380' '0410221aaa6380' '04203ed2955d80'
 '04102e1aaa6380']
(440,)
['20220130' '20220129' '20220128' '20220127' '20220126' '20220125'
 '20220124' '20220123' '20220122' '20220121' '20220120' '20220119'
 '20220118' '20220117' '20220116' '20220115' '20220114' '20220113'
 '202201

In [19]:
# Tomar todos los datos para un idsam en particular
vehiculo_test = datos_rutas

# Asegurarnos que todos los valores estan ordenados por fecha
vehiculo_test = vehiculo_test.sort_values(by='fechahoraevento')

# Tomar solo muestras de un periodo en particular
dia = vehiculo_test[(vehiculo_test['fechahoraevento'] > "2021-12-31") & (vehiculo_test['fechahoraevento'] < "2023-01-01")]
minx, miny, maxx, maxy = gpd.GeoDataFrame(dia, geometry=gpd.points_from_xy(dia.longitude,dia.latitude),crs="EPSG:4326").total_bounds

print("Esquinas de la ruta para 0145: ",minx, miny, maxx, maxy)

Esquinas de la ruta para 0145:  -57.6357 -25.5205 -57.3529 -25.1684


Ahora tratemos de visualizar que paso por día con esos lectores

In [20]:
fechasstr = datos_rutas['fechastr'].unique()
DIAS = ['Domingo','Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']

for fechastr in fechasstr:
    continue
    imgpath = './graphs/'+fechastr+'.png'
    
    if(os.path.isfile(imgpath)):
        continue
    
    # print(imgpath)
    
    dia_test = dia[(dia[['fechastr']] == fechastr).all(axis=1)] 

    fecha_linda = fechastr[:4]+'-'+fechastr[4:6]+'-'+fechastr[6:8]
    fecha_linda = fecha_linda + " " + DIAS[int(datetime.datetime.strptime(fechastr, '%Y%m%d').strftime("%w"))]
    
    latlon_dia = gpd.GeoDataFrame(dia_test, geometry=gpd.points_from_xy(dia_test.longitude,dia_test.latitude),crs="EPSG:4326")
    try:
        mapa_dia_plot = latlon_dia.plot(figsize=(9, 9),legend=False,marker='+',markersize=20, column='idsam')
        mapa_dia_plot.set(title='Datos '+fecha_linda+' para el idrutaestacion 0145')
        cx.add_basemap(mapa_dia_plot, crs="EPSG:4326",source=cx.providers.Stamen.Toner)
        mapa_dia_plot.set_xlim(xmin=minx, xmax=maxx)
        mapa_dia_plot.set_ylim(ymin=miny, ymax=maxy)
        plt.axis('equal')
        plt.savefig(imgpath,dpi=300)
        plt.close()
    except:
        continue
 

# print("Graficos generados")

Ahora que tenemos cierta idea de cual es el moviento de cada lector, podemos comenzar a crear conjeturas de como podemos generar una trayectoria.
- Uno de los aprendizajes que nos dejan estas imagenes es que un lector no necesariamente esta unido a una sola ruta; un lector puede, durante el día o durante la semana, cambiar de ruta.
- Una misma linea cubre rutas diferentes; pero estas otras rutas son cubiertas con frecuencias diferentes. Casi puede notarse una trayectoria principal y otra secundaria(s); cuando el lector debe cubrir una ruta adicional.
- Las rectas que vamos a dibujar deberan ser por día y por idsam; y rectas que no se ajusten a lo normal deberan ser agrupadas aparte o descartadas. Esto puede generar posiblemente que un idrutaestacion genere como minimo una cantidad de rutas asignadas + 1


Como notas valiosas sobre las cuales vamos a trabajar ahora dejo dos links:
- [Exploring Trip Fuel Consumption by Machine Learning from GPS and CAN Bus Data](https://doi.org/10.11175/easts.11.906)
- [Nivel de exactitud (Accuracy) que agrega cada digito de un valor GPS](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude)

In [21]:
vehiculo_random = datos_rutas.sample(1).values[0]
# print(vehiculo_random)
# vehiculo_random = ['0408211a846280',('2022-02-16 04:47:48'), -25.5071, -57.3592, 145,'20220216']
vehiculo_test = datos_rutas[(datos_rutas[['idsam']] == vehiculo_random[0]).all(axis=1)] 
vehiculo_test = vehiculo_test[(vehiculo_test[['fechastr']] == vehiculo_random[5]).all(axis=1)] 
vehiculo_test = vehiculo_test.sort_values(by='fechahoraevento')
vehiculo_test

,idsam,fechahoraevento,latitude,longitude,idrutaestacion,fechastr
269291,041c1ce24e6180,2022-11-05 09:17:56,-25.3464,-57.6344,145,20221105
269277,041c1ce24e6180,2022-11-05 09:24:10,-25.3353,-57.6250,145,20221105
269272,041c1ce24e6180,2022-11-05 09:24:46,-25.3331,-57.6255,145,20221105
269261,041c1ce24e6180,2022-11-05 09:28:12,-25.3334,-57.6198,145,20221105
269254,041c1ce24e6180,2022-11-05 09:30:42,-25.3339,-57.6137,145,20221105
...,...,...,...,...,...,...
267383,041c1ce24e6180,2022-11-05 23:09:18,-25.4275,-57.4512,145,20221105
267382,041c1ce24e6180,2022-11-05 23:09:18,-25.4275,-57.4512,145,20221105
267379,041c1ce24e6180,2022-11-05 23:13:00,-25.4416,-57.4425,145,20221105
267376,041c1ce24e6180,2022-11-05 23:13:22,-25.4420,-57.4422,145,20221105


Ahora vamos a agregar la librerias con las que vamos a empezar a dibujar los trayectos por día por idsam

In [27]:
import osmnx as ox
import networkx as nx

ox.settings.log_console=True
ox.settings.use_cache=True

mode = 'drive' # 'drive', 'bike', 'walk'
optimizer = 'time' # length
place = 'Asuncion, Paraguay'
graph = ox.graph_from_place(place, network_type = mode)
print(graph)

# vehiculo_ejemplo = vehiculo_test.head(5)
vehiculo_ejemplo = vehiculo_test
_anterior = vehiculo_ejemplo.iloc[0]
start_latlng = (_anterior.latitude,_anterior.longitude)

vehiculo_ejemplo = vehiculo_ejemplo.iloc[1: , :]
shortest_route_map_flag = False

for indice, punto in vehiculo_ejemplo.iterrows():
    
    end_latlng = (punto.latitude , punto.longitude)
    
    orig_node = ox.distance.nearest_nodes(graph, start_latlng[1], start_latlng[0])
    dest_node = ox.distance.nearest_nodes(graph, end_latlng[1], end_latlng[0])
    
    shortest_route = nx.shortest_path(graph,orig_node, dest_node, weight=optimizer)
    if(len(shortest_route)<2):
        continue
        
    if(shortest_route_map_flag==False):
        shortest_route_map = ox.plot_route_folium(graph, shortest_route, tiles='openstreetmap')
        shortest_route_map_flag=True
    else:
        shortest_route_map = ox.plot_route_folium(graph, shortest_route,route_map=shortest_route_map, tiles='openstreetmap')


    start_latlng = (punto.latitude , punto.longitude)
    _anterior=punto
    

shortest_route_map

MultiDiGraph with 9823 nodes and 26355 edges
